In [1]:
!pip install transformers
!pip install simpletransformers

English Data

In [2]:
import pandas as pd
df=pd.read_csv('/content/eng_3_train.tsv', delimiter='\t') 
df=df[["text", "category"]]
df = df.dropna()
df_dev=pd.read_csv('/content/eng_3_dev.tsv', delimiter='\t') 
df_dev=df_dev[["text", "category"]]
df_dev = df_dev.dropna()
df_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'text'}) 
df_test = df_test.dropna()

In [3]:
tedf=pd.read_csv('/content/tam_eng_train.tsv', delimiter='\t') 
tedf=tedf[["text", "category"]]
tedf = tedf.dropna()
tedf_dev=pd.read_csv('/content/tam_eng_dev.tsv', delimiter='\t') 
tedf_dev=tedf_dev[["text", "category"]]
tedf_dev = tedf_dev.dropna()
tedf_test=pd.read_csv('/content/tam_eng_test.tsv', delimiter='\t') 
tedf_test = tedf_test.dropna()

### Mixing english and tamil translated to english data

In [4]:
train = pd.concat([df, tedf])
valid = pd.concat([df_dev,tedf_dev])
test = pd.concat([tedf_test,df_test])

In [5]:
print(len(train))
print(len(valid))
print(len(test))

5817
1456
1819


In [6]:
concatenated = pd.concat([train, test,valid])

import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

df = concatenated

X = df['text']
y = df['category']

testsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)

train_val_idx, test_idx = next(testsplit.split(X, y))

X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]


validsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)

train_idx, val_idx = next(validsplit.split(X_train_val, y_train_val))

X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]

train_df = pd.DataFrame({'text': X_train, 'category': y_train})
val_df = pd.DataFrame({'text': X_val, 'category': y_val})
test_df = pd.DataFrame({'text': X_test, 'category': y_test})

print(f'Training set shape: {train_df.shape}')
print(f'Validation set shape: {val_df.shape}')
print(f'Test set shape: {test_df.shape}')


Training set shape: (5454, 2)
Validation set shape: (1819, 2)
Test set shape: (1819, 2)


In [7]:
print(train_df['category'].value_counts())
print(val_df['category'].value_counts())
print(test_df['category'].value_counts())

Non-anti-LGBT+ content    4711
Homophobic                 598
Transphobic                145
Name: category, dtype: int64
Non-anti-LGBT+ content    1571
Homophobic                 199
Transphobic                 49
Name: category, dtype: int64
Non-anti-LGBT+ content    1571
Homophobic                 200
Transphobic                 48
Name: category, dtype: int64


In [8]:
train_df['category'] = train_df['category'].replace('Non-anti-LGBT+ content', 'ally')
val_df['category'] = val_df['category'].replace('Non-anti-LGBT+ content', 'ally')
test_df['category'] = test_df['category'].replace('Non-anti-LGBT+ content', 'ally')

In [9]:
train_df['category'] = train_df['category'].replace('ally', 0)
train_df['category'] = train_df['category'].replace('Homophobic', 1)
train_df['category'] = train_df['category'].replace('Transphobic', 2)

In [10]:
val_df['category'] = val_df['category'].replace('ally', 0)
val_df['category'] = val_df['category'].replace('Homophobic', 1)
val_df['category'] = val_df['category'].replace('Transphobic', 2)

In [11]:
test_df['category'] = test_df['category'].replace('ally', 0)
test_df['category'] = test_df['category'].replace('Homophobic', 1)
test_df['category'] = test_df['category'].replace('Transphobic', 2)

In [12]:
train_df=train_df[["text", "category"]]
train_df = train_df.rename(columns={'text': 'text', 'category': 'labels'})
val_df=val_df[["text", "category"]]
val_df = val_df.rename(columns={'text': 'text', 'category': 'labels'})
test_df=test_df[["text", "category"]]
test_df = test_df.rename(columns={'text': 'text', 'category': 'labels'})

In [13]:
!pip install emoji

In [14]:
import emoji
train_df['text'] = train_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
val_df['text'] = val_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
test_df['text'] = test_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

## Training Model with Englis Data

In [16]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 3,
             "max_seq_length": 128,
             "train_batch_size": 32,
             "eval_batch_size": 32,
             "logging_steps": 50,
             "save_steps": 2000,
             "learning_rate": 2e-5,
             "manual_seed": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "xlnet", "xlnet-base-cased",
    num_labels=3,
    args=train_args
)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [17]:
# Train the model
model.train_model(train_df)

  0%|          | 0/5454 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/171 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/171 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/171 [00:00<?, ?it/s]

(513, 0.30294804630499717)

Validation Set


In [18]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')
result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score)
result

  0%|          | 0/1819 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/57 [00:00<?, ?it/s]

{'mcc': 0.5755387008014897,
 'f1': 0.6438588509546761,
 'acc': 0.9021440351841671,
 'eval_loss': 0.2778120953262898}

Testing With English Data

In [19]:
test_df

,text,labels
547,How u will decide the nature,0
2903,"Interviewer very worse communication, you are ...",0
49,Androgen insentivity syndrome. Females who loo...,0
1480,You are from He'll. ... Let God intervene and ...,0
1191,All transgenders,0
...,...,...
1395,Only the Holy Word of God ...will be written i...,0
1371,Thanks for supporting sir,0
326,"A year ago, when my stepdad was there, he crie...",0
412,bible is unscientific. proven in Supreme Court...,0


In [20]:
test = test_df['text'].tolist()
labels = test_df['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,Bro it is very bad,1,0
1,All the pundits who come here and support this...,1,0
2,Che Kanaravi ..,1,0
3,It is because of these transgender dogs that o...,2,0
4,I recognize the sacred relationship of my pare...,1,0
...,...,...,...
147,That's y no rains...,1,0
148,Unna patha le vandi vartu,1,0
149,Anywhere. These people,0,1
150,Bunda,1,0


In [21]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.94      0.97      0.96      1571
  homophobic       0.71      0.62      0.66       200
 transphobic       0.73      0.33      0.46        48

    accuracy                           0.92      1819
   macro avg       0.79      0.64      0.69      1819
weighted avg       0.91      0.92      0.91      1819



In [52]:
o_test=pd.read_csv('/content/chart gpt -test set', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_3_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

,"Love is love, no matter who you are or who you love. - Non-anti-LGBT+ content"
0,"Being gay is a sin, and those who engage in ho..."
1,Transgender people are confused and need psych...
2,I fully support the LGBT+ community and their ...
3,Marriage is between one man and one woman. Any...
4,Transgender people are brave and should be res...
5,Gay people are perverts who are destroying tra...
6,Transgender people deserve equal rights and pr...
7,"I don't believe in the transgender agenda, and..."
8,"Love is love, and everyone deserves to be happ..."
9,"Homosexuality is a lifestyle choice, not somet..."


In [ ]:
test_ml = pd.concat([o_test,e_test])

In [ ]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [ ]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [ ]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [ ]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,எனது கருத்துக்கு பதிலளித்த நீங்கள் இருவரும் இத...,1,0
1,"சத்திய நாராயணன் , இன்றும் மேற்கத்திய நாடுகளில்...",1,0
2,மனநோய் என்று நினைக்கிறேன். சென்று மருத்துவரை அ...,1,0
3,அவர்களை போலீசார் தண்டிக்க வேண்டும்,1,0
4,டைரக்டர் சார் ப்ளீஸ் கே தவம் ஆகுற மாரி படம் எட...,1,0
...,...,...,...
237,ரொம்ப நல்லவபோல நடிக்கறாள். பேசுகிறாள் காசுகொடு...,2,0
238,உங்கள் கருத்தை நான் ஏற்று கொள்கிறேன். ஓரினச் ச...,1,0
239,இந்தமாதி நாய்களை கண்ட இடத்தில் சுட்டுக்கொல்லனும்,2,0
240,தமிழில் கேட்ட அருமை விளக்கம். கலவியை(sex) இனப்...,0,1


In [ ]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.87      1.00      0.93      1579
  homophobic       0.40      0.02      0.04       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.87      1819
   macro avg       0.42      0.34      0.32      1819
weighted avg       0.80      0.87      0.81      1819



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with english and testing with tamil

In [22]:
o_test=pd.read_csv('/content/eng_tam_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_3_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [23]:
test_ml = pd.concat([o_test,e_test])

In [24]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [25]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [26]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [27]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,எனது கருத்துக்கு பதிலளித்த நீங்கள் இருவரும் இத...,1,0
1,"சத்திய நாராயணன் , இன்றும் மேற்கத்திய நாடுகளில்...",1,0
2,மனநோய் என்று நினைக்கிறேன். சென்று மருத்துவரை அ...,1,0
3,அவர்களை போலீசார் தண்டிக்க வேண்டும்,1,0
4,டைரக்டர் சார் ப்ளீஸ் கே தவம் ஆகுற மாரி படம் எட...,1,0
...,...,...,...
237,ரொம்ப நல்லவபோல நடிக்கறாள். பேசுகிறாள் காசுகொடு...,2,0
238,உங்கள் கருத்தை நான் ஏற்று கொள்கிறேன். ஓரினச் ச...,1,0
239,இந்தமாதி நாய்களை கண்ட இடத்தில் சுட்டுக்கொல்லனும்,2,0
240,தமிழில் கேட்ட அருமை விளக்கம். கலவியை(sex) இனப்...,0,1


In [28]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.87      1.00      0.93      1579
  homophobic       0.40      0.02      0.04       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.87      1819
   macro avg       0.42      0.34      0.32      1819
weighted avg       0.80      0.87      0.81      1819



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with english and testing with Hindi

In [29]:
o_test=pd.read_csv('/content/eng_hin_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_hin_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [30]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [31]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [32]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [33]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [34]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"आप दोनों जिन्होंने मेरी टिप्पणी का जवाब दिया, ...",1,0
1,"सत्य नारायणन , इसे आज भी पश्चिमी दुनिया में प्...",1,0
2,मुझे लगता है..यह मानसिक बीमारी है। जाओ और डॉक्...,1,0
3,पुलिस को उन्हें सजा देनी चाहिए,1,0
4,निर्देशक सर pls गे रिपेंट अगुरा मारी फिल्म एडुंगा,1,0
...,...,...,...
235,आप बौद्धिक रूप से बोलने के बारे में सोचते हैं ...,1,0
236,वह बहुत अच्छा काम करती है। मांचूर में डेढ़ साल...,2,0
237,मैं आपकी राय मानता हूं। अगर समलैंगिकता गलत है ...,1,0
238,इसी महीने कुत्तों को देखते ही गोली मार दी गई,2,0


In [35]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.87      1.00      0.93      1579
  homophobic       0.00      0.00      0.00       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.87      1819
   macro avg       0.29      0.33      0.31      1819
weighted avg       0.75      0.87      0.81      1819



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with english and testing with Telugu

In [36]:
o_test=pd.read_csv('/content/eng_tel_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_tel_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [37]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [38]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [39]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [40]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [41]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,నా వ్యాఖ్యకు ప్రత్యుత్తరం ఇచ్చిన మీరిద్దరూ దీన...,1,0
1,"సత్య నారాయణన్ , నేటికీ పాశ్చాత్య ప్రపంచంలో దీన...",1,0
2,నా అభిప్రాయం .. ఇది మానసిక వ్యాధి. వెళ్లి వైద్...,1,0
3,పోలీసులు వారిని శిక్షించాలి,1,0
4,డైరెక్టర్ సర్ ప్లీస్ గే పశ్చాత్తాపపడండి అగురా ...,1,0
...,...,...,...
235,మీరు తెలివిగా మాట్లాడాలని అనుకుంటున్నారు...,1,0
236,ఆమె చాలా చక్కగా నటిస్తుంది. మంచూర్‌లో డబ్బులు ...,2,0
237,నేను మీ అభిప్రాయాన్ని అంగీకరిస్తున్నాను. స్వలి...,1,0
238,ఈ నెలలో కుక్కలను చూసి కాల్చి చంపారు,2,0


In [42]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.87      1.00      0.93      1579
  homophobic       0.33      0.01      0.01       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.87      1819
   macro avg       0.40      0.33      0.31      1819
weighted avg       0.79      0.87      0.81      1819



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with english and testing with Malyalam

In [43]:
o_test=pd.read_csv('/content/eng_ml_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_ml_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [44]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [45]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [46]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [47]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [48]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,എന്റെ അഭിപ്രായത്തിന് മറുപടി നൽകിയ നിങ്ങൾ രണ്ടു...,1,0
1,"സത്യ നാരായണൻ , ഇന്നും പാശ്ചാത്യലോകത്ത് ഇത് പ്ര...",1,0
2,എനിക്ക് തോന്നുന്നു .. മാനസിക രോഗമാണ്. പോയി ഡോക...,1,0
3,പോലീസ് അവരെ ശിക്ഷിക്കണം,1,0
4,"സംവിധായകൻ സാർ സ്വവർഗാനുരാഗി പശ്ചാത്തപിക്കട്ടെ,...",1,0
...,...,...,...
236,നിങ്ങൾ ബുദ്ധിപരമായി സംസാരിക്കാൻ വിചാരിക്കുന്നു...,1,0
237,അവൾ വളരെ ഭംഗിയായി അഭിനയിക്കുന്നു. മഞ്ചൂരിൽ സർക...,2,0
238,താങ്കളുടെ അഭിപ്രായം ഞാൻ അംഗീകരിക്കുന്നു. സ്വവർ...,1,0
239,ഈ മാസം നായ്ക്കളെ കണ്ടപ്പോൾ വെടിവച്ചു,2,0


In [49]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.87      1.00      0.93      1579
  homophobic       0.25      0.01      0.01       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.87      1819
   macro avg       0.37      0.33      0.31      1819
weighted avg       0.78      0.87      0.81      1819



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
"Love is love, no matter who you are or who you love." - Non-anti-LGBT+ content
"Being gay is a sin, and those who engage in homosexual acts will go to hell." - Homophobic
"Transgender people are confused and need psychiatric help to get back to their birth sex." - Transphobic
"I fully support the LGBT+ community and their right to love who they want." - Non-anti-LGBT+ content
"Marriage is between one man and one woman. Anything else is unnatural and wrong." - Homophobic
"Transgender people are brave and should be respected for being true to themselves." - Non-anti-LGBT+ content
"Gay people are perverts who are destroying traditional values and corrupting our society." - Homophobic
"Transgender people deserve equal rights and protections under the law." - Non-anti-LGBT+ content
"I don't believe in the transgender agenda, and I won't participate in it." - Transphobic
"Love is love, and everyone deserves to be happy and free to express themselves." - Non-anti-LGBT+ content
"Homosexuality is a lifestyle choice, not something people are born with." - Homophobic
"I don't understand why transgender people feel the need to change their gender." - Transphobic
"I believe that marriage should only be between a man and a woman." - Homophobic
"The LGBT+ community should have the same rights and protections as everyone else." - Non-anti-LGBT+ content
"Transgender people are just trying to get attention and be different." - Transphobic
"I think that people should be able to love whoever they want, regardless of gender." - Non-anti-LGBT+ content
"Being gay is a mental disorder that needs to be cured." - Homophobic
"Transgender athletes should not be allowed to compete in sports with their identified gender." - Transphobic
"I fully support the LGBT+ community and their fight for equality." - Non-anti-LGBT+ content
"Gay people are going to hell and need to repent for their sins." - Homophobic
